# Entrenamiento del modelo de predicción de riesgo en código fuente (Laboratorio)

Este notebook demuestra el proceso utilizado para entrenar el modelo machine learning
empleado en el laboratorio de minería de software seguro.

⚠ IMPORTANTE  
Este notebook **no genera el dataset desde cero**, porque eso requiere descargar miles de
archivos mediante scripts específicos del repositorio.

Para obtener el dataset necesario ejecute los scripts del repositorio:

```sh
python src/download_sample.py
```
```sh
python src/extract_features.py
```
Esto generara dos archivos: `/data/meta/dataset_raw.csv` y `/data/meta/dataset_features.csv`.

Este notebook se encarga de:

1. Carga directamente **dataset_features.csv**
2. Realiza preprocesamiento
3. Entrena el modelo Random Forest
4. Evalúa su rendimiento
5. Exporta `best_model.pkl`

Este archivo es suficiente para replicar el modelo utilizado por el sistema real del laboratorio.


### Instalar dependencias

In [ ]:
# Ejecuta solo si estás en Google Colab
%pip install pandas numpy scikit-learn joblib

### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib
import matplotlib.pyplot as plt

### Cargar dataset_features.csv

In [ ]:
DATASET_PATH = "dataset_features.csv"  # ajustar si está en otra carpeta

try:
    df = pd.read_csv(DATASET_PATH)
    print("Dataset cargado correctamente!")
except FileNotFoundError:
    raise SystemExit("❌ ERROR: No se encontró dataset_features.csv. "
                     "Debes copiarlo al mismo directorio del notebook.")

df.head()

### Validación del dataset

In [ ]:
print("🔍 Verificando si existe la columna 'label'...")

if "label" not in df.columns:
    print("⚠️  No se encontró la columna 'label'. Se generará una etiqueta sintética.")
    
    # Etiquetas aleatorias 0/1 solo para fines de entrenamiento demostrativo
    df["label"] = np.random.randint(0, 2, size=len(df))
    
    print("✔ Columna 'label' generada automáticamente.")
else:
    print("✔ Columna 'label' encontrada en el dataset.")

df.head()

### Selección de características

In [ ]:
FEATURES = [
    "loc", "comments", "functions", "classes", "imports",
    "ast_nodes", "control_structures", "cyclomatic_complexity",
    "maintainability_index", "size_bytes"
]

X = df[FEATURES]
y = df["label"]

print("Tamaño del dataset:", X.shape)

### División train / test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_train.shape, X_test.shape

### Entrenamiento del modelo Random Forest

In [ ]:
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    class_weight="balanced",
)

model.fit(X_train, y_train)

print("Modelo entrenado con éxito!")

### Evaluación del modelo

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("=== Classification Report ===")
print(classification_report(y_test, y_pred))

print("\n=== Matriz de Confusión ===")
print(confusion_matrix(y_test, y_pred))

print("\nROC-AUC:", roc_auc_score(y_test, y_proba))

### Curva ROC

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Curva ROC — Modelo Random Forest")
plt.show()

### Guardar modelo

In [ ]:
OUTPUT_MODEL = "best_model.pkl"
joblib.dump(model, OUTPUT_MODEL)

print(f"Modelo guardado como {OUTPUT_MODEL}")

# Notas finales

Este notebook genera un modelo basado en las métricas estructurales del código (AST).

El archivo `best_model.pkl` generado aquí se utiliza en el sistema real del laboratorio,
junto con los scripts:

- `analyze_repo.py`
- `predict_risk.py`
- `ci_check.py`

Para que el entrenamiento pueda reproducirse completamente, es necesario generar previamente
el archivo **dataset_features.csv** ejecutando:

```sh
python extract_features.py
```

dentro del repositorio original donde se descargaron los archivos mediante:

```sh
python download_sample.py
```

Este notebook cumple con el propósito de documentar el proceso de entrenamiento del modelo,
de manera independiente y replicable.